In [149]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [ ]:
space_table = MyPyTable().load_from_file('input_data/space.csv')

space_table.remove_rows_with_missing_values()


for item in space_table.data:
    del item[0]
    del item[0]
    del item[0]
    del item[2]
del space_table.column_names[0]
del space_table.column_names[0]
del space_table.column_names[0]
del space_table.column_names[2]

min_diameter_bins, min_counts = myutils.discretization(space_table.get_column(space_table.column_names.index("estimated_diameter_min")))

for instance in space_table.data:
    for i, (lower, upper) in enumerate(min_diameter_bins):
        if lower <= instance[space_table.column_names.index("estimated_diameter_min")] <= upper:
            instance[space_table.column_names.index("estimated_diameter_min")] = i 
            break
max_diameter_bins, max_counts = myutils.discretization(space_table.get_column(space_table.column_names.index("estimated_diameter_max")))
for instance in space_table.data:
    for i, (lower, upper) in enumerate(max_diameter_bins):
        if lower <= instance[space_table.column_names.index("estimated_diameter_max")] <= upper:
            instance[space_table.column_names.index("estimated_diameter_max")] = i 
            break
velocity_bins, velocity_counts = myutils.discretization(space_table.get_column(space_table.column_names.index("relative_velocity")))
for instance in space_table.data:
    for i, (lower, upper) in enumerate(velocity_bins):
        if lower <= instance[space_table.column_names.index("relative_velocity")] <= upper:
            instance[space_table.column_names.index("relative_velocity")] = i 
            break
miss_bins, miss_counts = myutils.discretization(space_table.get_column(space_table.column_names.index("miss_distance")))
for instance in space_table.data:
    for i, (lower, upper) in enumerate(miss_bins):
        if lower <= instance[space_table.column_names.index("miss_distance")] <= upper:
            instance[space_table.column_names.index("miss_distance")] = i 
            break

y = space_table.get_column(space_table.column_names.index("is_hazardous"))  
X = [row[:space_table.column_names.index("is_hazardous")] + row[space_table.column_names.index("is_hazardous") + 1:] for row in space_table.data]

tree_classifier = MyDecisionTreeClassifier()
tree_classifier.fit(X,y)

header = space_table.column_names
tree_classifier.print_decision_rules(attribute_names=header)
tree_classifier.visualize_tree(
    "space_tree.dot", "space_tree", attribute_names=header
)
print(tree_classifier.tree)

IF relative_velocity == 0 AND miss_distance == 0 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 1 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 2 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 3 AND estimated_diameter_min == 0 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 3 AND estimated_diameter_min == 1 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 3 AND estimated_diameter_min == 2 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 3 AND estimated_diameter_min == 4 THEN class = FALSE.
IF relative_velocity == 0 AND miss_distance == 4 THEN class = FALSE.
IF relative_velocity == 1 AND miss_distance == 0 THEN class = FALSE.
IF relative_velocity == 1 AND miss_distance == 1 THEN class = FALSE.
IF relative_velocity == 1 AND miss_distance == 2 THEN class = FALSE.
IF relative_velocity == 1 AND miss_distance == 3 THEN class = FALSE.
IF relative_velocity == 1 AND miss_distance 